# Final Project - Data Wrangling with MongoDB

## Exploring the Open Street Maps Data for Disney World

For this project I will be exploring the data from the Open Street Maps database.
The file is provided in an XML format.

The Area I have chosen is around Disney World in Orlando, Fl. 
The data was downloaded from https://www.openstreetmap.org/export#map=13/28.3826/-81.5303

The size of my uncompressed osm file is 102 MB

In [22]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json
from collections import defaultdict
import pandas as pd


FILE = './DisneyWorld.osm'

In [34]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\.\t\r\n]')
lower_dot = re.compile(r'^([a-z]|_)*.([a-z]|_)*$')
numbers_only = re.compile('[^0-9]')
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
phonechars = re.compile(r'[(+).\-\. ]')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

# Examing the Data

## Problem Areas

Now we will begin going though the dataset to find potential problems with that data.


### Problem 1 - NonPrintable Chars in User Names
While trying to get a count of submitters I noticed that some users had unicode char in them. Notable "ó" in user 'Diógenes de Sinope'. I decided the best method of dealing with no printable chars was to remove them from the name and replace it with a ?. 
I did this with the remove_non_ascii function. The function is called as part of the count_submitters function. The count_submitters iterates through the data looking for 'user' field in the node tag.

In [3]:
#this dataset has users that have char outside of ASCII char set.
#Replacing non printable chars with a ?
def remove_non_ascii(text):
    return re.sub(r'[^\x00-\x7F]+','?', text)

In [4]:
def count_submitters(filename):
    users = {}
    for event, elem in ET.iterparse(filename, events=('start', )):
        #checks if user is in the attrib list for the element
        if 'user' in elem.attrib:
            #replaces non printable chars with ?
            user = remove_non_ascii(elem.attrib['user'])
            if user not in users:
                users[user] = 1
            else:
                users[user] += 1
    return users

In [5]:
submitter = count_submitters(FILE)
submitterDF = pd.Series(submitter).to_frame()
submitterDF.columns = ['Count']
submitterDF.sort_values('Count', ascending = False).head(10)

,Count
Brian@Brea,105299
NE2,82904
3yoda,74164
Mongo Poker,52738
Di?genes de Sinope,27023
epcotfan,24093
KindredCoda,15541
RobChafer,14662
Adam Martin,12621
Gulopine,11857


### Problem 2 - Postcodes of Different Formats and Incorrect Information

While trying to determine the zipcodes of my dataset, I noticed that some information was incorrect or not correctly formated.
I crafted a function that would Count the number of zipcodes in the file as well as clean them up so they are all uniform 5 chars in length. I also noticed 4 zipcodes that were in other areas of the city, so I actually submitted corrections to the OSM website correcting those issues. I then redownloaded the problem area with the correct changes. This leaves us with 9 distinct postcodes.

In [6]:
def count_postcodes(filename):
    postcodes = {}
    for event, elem in ET.iterparse(filename, events=('start', 'end')):
        if event == 'end':
            key = elem.attrib.get('k')
            if key == 'addr:postcode':
                postcode = clean_postcodes(elem.attrib.get('v'))
                if postcode[:5] not in postcodes:
                    postcodes[postcode[:5]] = 1
                else:
                    postcodes[postcode[:5]] += 1
    return postcodes

In [7]:
def clean_postcodes(zip):
    zip = numbers_only.sub('', zip)
    if len(zip) == 5:
        return zip
    else:
        return zip[:5]

In [8]:
postcodes = count_postcodes(FILE)
postcodesDF = pd.Series(postcodes).to_frame()
postcodesDF.columns = ['Count']
postcodesDF.sort_values('Count', ascending = False)

,Count
34746,95
34747,80
32837,66
32836,53
32821,48
34786,45
32830,43
32819,16
34741,5


### Problem 3 - Street Name Inconsistencies

The next problem to focus my attention on would be creating a method from cleaning up the street name and ensuring that clean correct data is returned to be put into the database

In [9]:
mapping = { "St": "Street","St.": "Street","Ave" : "Avenue","Ave." : "Avenue","Blvd" : "Boulevard","Blvd." : "Boulevard",
            "Dr" : "Drive","Dr." : "Drive","Ct" : "Court","Ct." : "Court","Pl" : "Place","Pl." : "Place","Sq" : "Square",
            "Sq." : "Square","Ln" : "Lane","Ln." : "Lane","Rd" : "Road","Rd." : "Road","Tr" : "Trail","Tr." : "Trail",
            "Pkwy" : "Parkway","Pkwy." : "Parkway","Cmns" : "Commons","Cmns." : "Commons","N." : "North","N" : "North",
            "W." : "West","W" : "West","S." : "South","S" : "South","E." : "East","E" : "East" }
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Way", "Stars","Circle", "Cove", "Highway"]

In [10]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r", errors="ignore")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


def update_name(name, mapping):
    nameArr = name.split(" ")
    for i in range(len(nameArr)):
        if nameArr[i] in mapping:
            nameArr[i] = mapping[nameArr[i]]
    name = " ".join(nameArr)
    return name

In [11]:
def check_names():
    st_types = audit(FILE)
    #pprint.pprint(dict(st_types))
    for st_type, ways in st_types.items():
        for name in ways:
            better_name = update_name(name, mapping)
            print(name, "=>", better_name)


In [12]:
check_names()

Broadway => Broadway
N John Young Pkwy => North John Young Parkway
Kyng's Heath Rd => Kyng's Heath Road
Rolling Oaks Blvd. => Rolling Oaks Boulevard
World Center Dr => World Center Drive
World Cennter Dr. => World Cennter Drive
Equestrian Dr. => Equestrian Drive
International Drive South => International Drive South
Celebration Blvd => Celebration Boulevard
Formosa Gardens Blvd => Formosa Gardens Boulevard
WorldQuest Blvd => WorldQuest Boulevard
Towne Center Blvd => Towne Center Boulevard
Polynesian Isle Blvd => Polynesian Isle Boulevard
Archfeld Blvd => Archfeld Boulevard
Rolling Oaks Blvd => Rolling Oaks Boulevard
Shiva Loop => Shiva Loop
State Rd 535 => State Road 535
State Road 535 => State Road 535


In [13]:
def audit_street_num(osmfile):
    osm_file = open(osmfile, "r", errors="ignore")
    street_num = []
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_number(tag):
                    if bool(re.search(r'[aA-zZ]', tag.attrib['v'])):
                        if tag.attrib['v'] not in street_num:
                            street_num.append(tag.attrib['v'])
    osm_file.close()
    return street_num

def is_street_number(elem):
    return (elem.attrib['k'] == "addr:housenumber")


In [15]:
street_nums = audit_street_num(FILE)
print(street_nums)

[]


Now that we have an empty array returned, we are sure that there are no incorrect housenumbers.

In [30]:
def print_values(osmfile, value):
    osm_file = open(osmfile, "r", errors="ignore")
    street_types = []
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == value:
                    if tag.attrib['v'] not in street_types:
                        street_types.append(tag.attrib['v'])
    osm_file.close()
    return street_types

print(print_values(FILE, 'addr:city'))

['Orlando', 'Lake Buena Vista', 'Celebration', 'Kissimmee', 'Windermere', 'Lake Buena Vista,', 'Bay Lake']


From this we can also see that we have some cities that need to be cleaned up as well. Specifically the one with the comma. We can send the values through the problem_char re to clean them up.

In [37]:
def clean_values(osmfile, value):
    osm_file = open(osmfile, "r", errors="ignore")
    street_types = []
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == value:
                    city = problemchars.sub("", tag.attrib['v'])
                    if city not in street_types:
                        street_types.append(city)
    osm_file.close()
    return street_types

print(clean_values(FILE, 'addr:city'))

['Orlando', 'Lake Buena Vista', 'Celebration', 'Kissimmee', 'Windermere', 'Bay Lake']


From this we can see that the easist way to normalize the cities is to use the problemchar regex and remove all charecters that match to it. We now have 6 distinct cities for the area.

# Problem 4 - Phone Number Issues
The next problem to focus my attention on would be the different phone numbers stored in the database. I will craft a function that will identify problem telephone numbers.

In [16]:
def audit_phone(osmfile, replace):
    osm_file = open(osmfile, "r", errors="ignore")
    phone_num = []
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_phone_number(tag):
                    newphone = clean_phone(tag.attrib['v'], replace)
                    if bool(re.search(r'[aA-zZ]', newphone)):
                        if newphone not in phone_num:
                            phone_num.append(newphone)
    osm_file.close()
    return phone_num

def is_phone_number(elem):
    return (elem.attrib['k'] == "phone")

def clean_phone(phone, char_replace):
    phone = phonechars.sub("",phone)
    if char_replace:
        if bool(re.search(r'[aA-zZ]', phone)):
            return replace_chars(phone)
    if phone.startswith('1'):
        return phone[1:]
    if phone.startswith('01'):
        return phone[2:]
    return phone

In [17]:
phone_nums = audit_phone(FILE, False)
print(phone_nums)

['407WDWPLAY', '407WDWCREW']


In [18]:
#So we have some phone numbers that have Char in them. Need to convert to numbers
def replace_chars(phone):
    try:
        translationdict = str.maketrans("abcdefghijklmnopqrstuvwxyz","22233344455566677778889999")
    except AttributeError:
        import string
        translationdict = string.maketrans("abcdefghijklmnopqrstuvwxyz","22233344455566677778889999")

    correct_phone = phone.lower().translate(translationdict)
    return correct_phone

In [19]:
#we call the same function with a true parameter now
#it will replace the letters in the phonenumber with the corresponding digit.
phone_nums = audit_phone(FILE, True)
print(phone_nums)

[]


Now that the phone numbers are cleaned up this means that the data should be ready to be shaped and proccesed

In [20]:
def count_tags(filename):
    tags = {}
    for event, elem in ET.iterparse(filename, events=('start', )):
        if elem.tag not in tags:
            tags[elem.tag] = 1
        else:
            tags[elem.tag] += 1
    return tags

In [21]:
def count_keys(filename):
    keys = {}
    for event, elem in ET.iterparse(filename, events=('start', 'end')):
        if event == 'end':
            key = elem.attrib.get('k')
            if key:
                if key not in keys:
                    keys[key] = 1
                else:
                    keys[key] += 1
    return keys

# Converting to json

Now that we have a plan of attack for how we will clean up the data while it is being import. We can begin by crafting the shape_element function. For this project we are only concerned with proccesing if it is a node or way. The shape element function takes the element object passed to it when it is called.  

In [190]:
def shape_element(element):
    if element.tag == "node":
        #creates a basic model for the data
        node = {
            "id": element.attrib['id'],
            "type": "node",
            "pos": None,
            "created" : {
                "changeset": None,
                "user": None,
                "version": None,
                "uid": None,
                "timestamp": None
            }
        }
        #iterates through the fields of the CREATER array and adds the to the node
        for i in CREATED:
            if i == 'user':
                node['created'].update({i : remove_non_ascii(element.attrib[i])})
            else:
                node['created'].update({i : element.attrib[i]})
        node['pos'] = [str(float(element.attrib['lat'])), str(float(element.attrib['lon']))]
        #Iterates through all the child elements of the main element. 
        for child in element:
                #checks if the key is all lower case
                if lower.search(child.attrib['k']):
                    if child.attrib['k'] == 'phone':
                        #if it is a phone number it calls the clean_phone function
                        node[child.attrib['k']] = clean_phone(child.attrib['v'], True)
                    else:
                        node[child.attrib['k']] = child.attrib['v']
                #splits the key on the ':' char and checks if the len is 2. if greater than 2 we ignore it.
                elif len(child.attrib['k'].split(':')) == 2:
                    arr = child.attrib['k'].split(':')
                    if arr[0] == 'addr':
                        if 'address' not in node:
                            node['address'] = {}
                        if arr[1] == 'street':
                            #calls the update_name fuction which returns a cleaned up street address
                            node['address'].update({arr[1] : update_name(child.attrib['v'], mapping)})
                        elif arr[1] == 'state':
                            #For state, if takes the first 2 char of the word and returns them upper. ie FL
                            node['address'].update({arr[1] : child.attrib['v'][:2].upper()})
                        elif arr[1] == 'city':
                            #removes the problem charecters in the value.
                            node['address'].update({arr[1] : problemchars.sub("", child.attrib['v'])})
                        else:
                            node['address'].update({arr[1] : child.attrib['v']})
                    elif arr[0] == 'name':
                        None
                    else:
                        node[child.attrib['k'].split(':')[0]] = {child.attrib['k'].split(':')[1] : child.attrib['v']}
                elif problemchars.search(child.attrib['k']):
                    None
                else:
                    None
        return node
    #Most of the same logic is there for this type of element. 
    elif element.tag == 'way':
        node = {
            "id" : element.attrib['id'],
            "type": "way",
            "created" : {
                "changeset": None,
                "user": None,
                "version": None,
                "uid": None,
                "timestamp": None
            }
        }
        for i in CREATED:
            if i == 'user':
                node['created'].update({i : remove_non_ascii(element.attrib[i])})
            else:
                node['created'].update({i : element.attrib[i]})
        for child in element:
            if child.tag == 'nd':
                if 'node_refs' not in node:
                    node['node_refs'] = []
                node['node_refs'].append(child.attrib['ref'])
            else:
                if lower.search(child.attrib['k']):
                    if child.attrib['k'] == 'phone':
                        node[child.attrib['k']] = clean_phone(child.attrib['v'], True)
                    else:
                        node[child.attrib['k']] = child.attrib['v']
                elif len(child.attrib['k'].split(':')) == 2:
                    arr = child.attrib['k'].split(':')
                    if arr[0] == 'addr':
                        if 'address' not in node:
                            node['address'] = {}
                        if arr[1] == 'street':
                            node['address'].update({arr[1] : update_name(child.attrib['v'], mapping)})
                        elif arr[1] == 'state':
                            node['address'].update({arr[1] : child.attrib['v'][:2].upper()})
                        elif arr[1] == 'city':
                            node['address'].update({arr[1] : problemchars.sub("", child.attrib['v'])})
                        else:
                            node['address'].update({arr[1] : child.attrib['v']})
                    elif arr[0] == 'name':
                        None
                    else:
                        node[child.attrib['k'].split(':')[0]] = {child.attrib['k'].split(':')[1] : child.attrib['v']}
                elif problemchars.search(child.attrib['k']):
                    None
                else:
                    None
        return node
    else:
        return None

In [191]:
def process_map(file_in, pretty):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    number_to_process = 0
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return print("Your File has been proccessed and exported as DisneyWorld.osm.json")

Finally we can call the process_map function. This will work through the file and call the shape element for each element returned by the iterparse function out of the element tree. This must be called with pretty turned off. Pymongo is not set up to handle the pretty formet.


In [192]:
process_map(FILE, False)

Your File has been proccessed and exported as DisneyWorld.osm.json


## Importing to MongoDB
Now that we have the processed json file we can begin the process of importing it into MongoDB

In [208]:
import pymongo
from pymongo import MongoClient

def insert_line(line, db):
    db.disneyworld.insert(line)

def insert_to_mongodb(filename, db):
    with open(filename) as f:
        for line in f:
            data = json.loads(line)
            insert_line(data,db)

In [209]:
client = MongoClient("mongodb://localhost:27017")
client.drop_database("disneyworld")
db = client.disneyworld

insert_to_mongodb("./DisneyWorld.osm.json", db)
print(db.disneyworld.find_one())

/Users/davidmanasco/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


{'type': 'node', 'pos': ['28.4277676', '-81.4169626'], 'created': {'version': '3', 'user': 'Eckhart W?rner', 'uid': '2675', 'timestamp': '2015-02-12T18:57:22Z', 'changeset': '28802602'}, '_id': ObjectId('5a8b9af19ffbe50758192396'), 'id': '16580374'}


In [195]:
db.disneyworld.count()

514455

Looks like we have 514,455 records succesfully imported into the disneyworld database via pymongo.

In [196]:
#count number of nodes
db.disneyworld.find({'type':'node'}).count()

470882

In [197]:
#count number of ways
db.disneyworld.find({'type':'way'}).count()

43372

In [198]:
#number of unique contributers to the dataset
len(db.disneyworld.distinct('created.user'))

411

In [214]:
#find top 5 users for the dataset
top_user = db.disneyworld.aggregate([{"$group":{"_id":"$created.user", "count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":5}])
for line in top_user:
    print(line)

{'count': 105159, '_id': 'Brian@Brea'}
{'count': 82623, '_id': 'NE2'}
{'count': 74148, '_id': '3yoda'}
{'count': 52343, '_id': 'Mongo Poker'}
{'count': 27017, '_id': 'Di?genes de Sinope'}


In [229]:
#find basic percentages of different types of users in the database
print("Percentage of items by top submitter (Brian@Brea) - "+"{0:.2f}".format(db.disneyworld.find({"created.user":"Brian@Brea"}).count()/db.disneyworld.count()*100)+"%")
print("Percentage of top two sumbitters (Brian@Brea, NE2) - "+"{0:.2f}".format(db.disneyworld.find({"created.user": { "$in" : ["Brian@Brea", "NE2"]}}).count()/db.disneyworld.count()*100)+"%")
print("Percentage of top five sumbitters - "+"{0:.2f}".format(db.disneyworld.find({"created.user": { "$in" : ["Brian@Brea", "NE2", "3yoda", "Mongo Poker", "Di?genes de Sinope"]}}).count()/db.disneyworld.count()*100)+"%")

Percentage of items by top submitter (Brian@Brea) - 20.44%
Percentage of top two sumbitters (Brian@Brea, NE2) - 36.50%
Percentage of top five sumbitters - 66.34%


From the above query results, we can see that the top 5 contributors have over 66% of the total updates to the database. This tells me that ordinary users are not inclined to contribute to the database. It seems that leaving the majority of the data up to only 5 users can lead to data integrity issues, as well as a myriad of other problems, like multiple entries for the same thing.

In [200]:
#find all theme_parks within the dataset
themeParks = db.disneyworld.find({"tourism": "theme_park", "name" : {"$exists" : "true"}}, {"name": 1, "operator" : 1, "_id" : 0})
themeParks = themeParks.sort([("operator", 1), ("name", 1)])
for row in themeParks:
    print(row)

{'operator': 'Disney Parks and Resorts', 'name': "Disney's Animal Kingdom"}
{'operator': 'Disney Parks and Resorts', 'name': "Disney's Blizzard Beach"}
{'operator': 'Disney Parks and Resorts', 'name': "Disney's Hollywood Studios"}
{'operator': 'Disney Parks and Resorts', 'name': "Disney's Typhoon Lagoon"}
{'operator': 'Disney Parks and Resorts', 'name': 'Epcot'}
{'operator': 'Disney Parks and Resorts', 'name': 'Epcot'}
{'operator': 'Disney Parks and Resorts', 'name': 'Magic Kingdom'}
{'operator': 'Fun Spot America Theme Parks', 'name': 'Fun Spot America - Kissimmee'}
{'operator': 'SeaWorld Parks & Entertainment', 'name': 'SeaWorld Orlando'}
{'operator': 'SeaWorld Parks and Entertainment', 'name': 'Aquatica'}
{'operator': 'SeaWorld Parks and Entertainment', 'name': 'Discovery Cove'}


In [201]:
#find all attractions within the dataset
attractions = db.disneyworld.find({"tourism": "attraction", "name" : {"$exists" : "true"}}, {"name": 1, "_id":0})
for row in attractions:
    print(row)

{'name': 'Liberty Bell'}
{'name': 'Fort Langhorn'}
{'name': 'Elephants'}
{'name': 'Elephants'}
{'name': 'Flamingos'}
{'name': 'Lions'}
{'name': 'Hippos'}
{'name': 'Hippos'}
{'name': 'Nile Crocodiles'}
{'name': 'Gorillas'}
{'name': 'Gorillas'}
{'name': 'Okapi'}
{'name': 'Black Rhinoceros'}
{'name': 'Aviary'}
{'name': 'Bongo'}
{'name': 'Maharajah Jungle Trek'}
{'name': 'Reflections of China'}
{'name': "Harper's Mill"}
{'name': 'Gran Fiesta Tour Starring The Three Caballeros'}
{'name': "Ariel's Grotto"}
{'name': 'Storm Slides'}
{'name': 'Humunga Kowabunga'}
{'name': 'Keelhaul Falls'}
{'name': 'Mayday Falls'}
{'name': 'Gangplank Falls'}
{'name': 'Ketchadiee Creek'}
{'name': 'Bay Slides'}
{'name': 'Castaway Creek'}
{'name': 'Fantasmic!'}
{'name': 'Character Spot'}
{'name': 'Journey Into Imagination With Figment'}
{'name': 'Imageworks: The "What If" Labs'}
{'name': 'Living With The Land'}
{'name': 'Turtle Talk with Crush'}
{'name': 'United Kingdom Pavilion'}
{'name': 'Canada Pavilion'}
{'nam

In [202]:
#find rollercoasters in database
rollerCoasters = db.disneyworld.find({"tourism" : "attraction", "attraction" : "roller_coaster"}, {"name" : 1, "_id": 0})
for line in rollerCoasters:
    print(line)

{'name': "Rock 'n' Roller Coaster Starring Aerosmith"}
{'name': 'Space Mountain'}


In [203]:
#find out what is under cunstruction in the database.
underConstruction = db.disneyworld.find({"landuse" : "construction", "name" : { "$exists": "true", "$ne": "null" }}, {"name" : 1, "_id" : 0})
for line in underConstruction:
    print(line)

{'name': 'Disney Riviera Resort'}
{'name': 'Old Port Royale Renovation (2019)'}
{'name': 'Disney Skyliner Caribbean Beach Station (2019)'}
{'name': 'Disney Skyliner Epcot Station (2019)'}
{'name': 'Parking Lot Expansion'}
{'name': 'Star Wars Hotel-Coming Soon'}
{'name': 'Guardians of the Galaxy Attraction-Coming Soon'}
{'name': 'Toy Story Land (Summer 2018)'}
{'name': 'Disney Skyliner Hollywood Studios Station (2019)'}
{'name': "Mickey and Minnie's Runaway Railway-2019"}
{'name': 'Wolfgang Puck Bar & Grill'}
{'name': 'Jaleo by José Andrés'}
{'name': 'NBA Experience'}
{'name': 'Wine Bar George (2018)'}
{'name': 'Third Parking Garage'}
{'name': 'Ratatouille Attraction Coming Soon'}
{'name': "Star Wars Galaxy's Edge (2019)"}


In [204]:
#find shops on Main Street in Magic Kingdom
mainstreet = db.disneyworld.find({"address.street" : "Main Street", "name" : { "$exists": "true", "$ne": "null" }}, {"name": 1, "_id":0})
for line in mainstreet:
    print(line)

{'name': 'Harmony Barber Shop'}
{'name': 'The Plaza Restaurant'}
{'name': 'Main Street Bakery'}
{'name': "Casey's Corner"}
{'name': 'Disney Clothiers'}
{'name': 'Main Street Confectionery'}
{'name': 'Main Street Gallery'}
{'name': 'Uptown Jewelers'}
{'name': 'Crystal Arts'}
{'name': 'Plaza Ice Cream Parlor'}
{'name': "Tony's Town Square Restaurant"}
{'name': 'The Emporium'}


In [189]:
#find all fast_food restauraunts in the dataset
fastfood = db.disneyworld.find({"amenity" : "fast_food", "name" : { "$exists": "true", "$ne": "null" }}, {"name": 1, "_id":0})
for line in fastfood:
    print(line)

{'name': 'Lotus Blossom Cafe'}
{'name': 'Sleepy Hollow'}
{'name': 'The Mara'}
{'name': 'Cookes of Dublin'}
{'name': 'Yorkshire County Fish Shop'}
{'name': 'Sunshine Seasons'}
{'name': "McDonald's"}
{'name': 'KFC'}
{'name': 'Tangierine Cafe'}
{'name': 'Subway'}
{'name': "Dunkin' Donuts; Baskin-Robbins"}
{'name': "Papa John's"}
{'name': "McDonald's"}
{'name': 'Burger King'}
{'name': "Casey's Corner"}
{'name': "Cici's Pizza"}
{'name': 'Subway'}
{'name': 'Flippers Pizza'}
{'name': 'Panera Bread'}
{'name': 'Subway'}
{'name': 'Firehouse Subs'}
{'name': 'Little Caesars Pizza'}
{'name': 'China King'}
{'name': 'Chipotle'}
{'name': "Good's Food To Go"}
{'name': 'Sassagoula Floatworks & Food Factory'}
{'name': 'Earl of Sandwich'}
{'name': 'Wolfgang Puck Express'}
{'name': 'The Paddock Grill'}
{'name': 'Riverside Mill Food Court'}
{'name': 'Meadow Depot'}
{'name': 'Roaring Fork'}
{'name': 'Landscape of Flavors'}
{'name': "Peevy's Polar Pipeline"}
{'name': 'Splash Grill'}
{'name': 'The Fountain'}
{

In [205]:
#find all railway or monorail stations in the dataset
stations = db.disneyworld.find({"railway" : "station", "name" : { "$exists": "true", "$ne": "null" }}, {"name": 1, "_id":0})
for line in stations:
    print(line)

{'name': 'Express Line: Magic Kingdom'}
{'name': 'Resort Line: Magic Kingdom'}
{'name': 'Resort Line: The Grand Floridian Resort'}
{'name': 'Resort Line: The Polynesian Village'}
{'name': 'Resort Line: Ticket and Transportation Center'}
{'name': 'Express Line: Ticket and Transportation Center'}
{'name': 'Epcot Line: Ticket and Transportation Center'}
{'name': 'Resort Line: The Contemporary Resort'}
{'name': 'Main Street, U.S.A. Station'}
{'name': 'Harambe Station'}
{'name': 'Conservation Station'}
{'name': 'Epcot Monorail Station'}
{'name': 'Fantasyland Station'}
{'name': 'Frontierland Station'}
{'name': 'People Mover'}
{'name': 'Main Street Station'}


In [236]:
hotels = db.disneyworld.find({"tourism" : "hotel", "name" : { "$exists": "true", "$ne": "null" }}, {"name": 1, "_id":0})
print("There are "+str(hotels.count())+ " hotels within the dataset")
for line in hotels.sort("name", 1):
    print(line)

There are 165 hotels within the dataset
{'name': 'Allure Resort International Drive Orlando'}
{'name': 'B Resort & Spa'}
{'name': 'BEST WESTERN Lake Buena Vista Resort Hotel'}
{'name': 'BEST WESTERN PLUS Orlando Convention Center Hotel'}
{'name': "Bay Lake Tower at Disney's Contemporary Resort"}
{'name': 'Blue Inn'}
{'name': 'Bluegreen Fountains Resort'}
{'name': 'Buena Vista Palace'}
{'name': 'Buena Vista Suites'}
{'name': 'Building 19 Vacation Village At Parkway'}
{'name': 'Caribe Royale All Suite Hotel & Convention Center'}
{'name': 'Celebration Suites'}
{'name': 'Champions World Resort'}
{'name': 'Claremont Inn'}
{'name': 'Claremont Kissimmee Hotel'}
{'name': 'Clarion Inn & Suites At International Drive'}
{'name': 'Clarion Inn Lake Buena Vista'}
{'name': 'Clarion Suites Maingate'}
{'name': 'Classic Worldwide Hotel'}
{'name': 'Comfort Inn'}
{'name': 'Comfort Inn Maingate'}
{'name': 'Comfort Suites Maingate East'}
{'name': 'Confortable Inn'}
{'name': 'Continental Plaza Hotel Kissimme

In [243]:
#returns the top 5 cuisines for restaurants in the dataset
for i in db.disneyworld.aggregate([{"$match":{"amenity":{"$exists":1}, "amenity":"restaurant",
                                    "cuisine" : { "$exists": "true", "$ne": "null" }}}, 
                                   {"$group":{"_id":"$cuisine", "count":{"$sum":1}}},{"$sort":{"count":-1}},
                                   {"$limit" : 5}]):
    print(i)

{'count': 31, '_id': 'american'}
{'count': 10, '_id': 'pizza'}
{'count': 8, '_id': 'mexican'}
{'count': 7, '_id': 'italian'}
{'count': 6, '_id': 'chinese'}


# Additional Improvements

The biggest thing issue I discovered while I was querying the database is that there are multiple documents for the same items. This leads to inacurate counts as well as confusing data. One example is there are two "Epcot"s in the dataset, yet only one exists in real life.

In [213]:
for i in db.disneyworld.find({"name" : "Epcot"}, {"node_refs": 0, "_id" : 0}):
    print(i)

{'tourism': 'theme_park', 'website': 'https://disneyworld.disney.go.com/destinations/epcot/', 'start_date': '1982-10-01', 'wikipedia': 'en:Epcot', 'name': 'Epcot', 'id': '449024190', 'alt_name': 'Experimental Prototype Community Of Tomorrow', 'designation': 'Walt Disney World Resort', 'address': {'state': 'FL', 'housenumber': '200', 'street': 'Epcot Center Drive', 'postcode': '32821', 'city': 'Lake Buena Vista'}, 'type': 'way', 'wheelchair': 'yes', 'old_name': 'Epcot Center', 'operator': 'Disney Parks and Resorts', 'created': {'version': '4', 'user': 'Mongo Poker', 'uid': '4256778', 'timestamp': '2018-01-02T22:29:14Z', 'changeset': '55112502'}, 'opening_hours': 'Mon-Sun (Epcot park hours vary from day to day. Future World and World Showcase usually open at different times. For current park hours visit https://disneyworld.disney.go.com/calendars/)'}
{'tourism': 'theme_park', 'website': 'https://disneyworld.disney.go.com/destinations/epcot/', 'wikipedia': 'en:Epcot', 'name': 'Epcot', 'id

You can see from the two returned from that query that they are very similar. So it would appeard the biggest hurdle of this dataset is accurate and complete data. 

The biggest hurdle with trying to address multiple documents for the same location, would be that it would be extremely time consuming to check which document is the most accurate and what you need to merge between the two files. There arent many ways to programatically merge documents because it requires a human touch.

# Conclusion

The conclusions that I have drawn is that there are definitely power users who account from over half of all the map edits. This can lead to problems when it comes to data acuracy and cleanliness. For the most part the data was pretty cleaned and ready for database import. There were a few examples where I had to pass the value through a function to return a cleaned format of the value. This was all very easy to accomplish programatically. The area around Disney World is absolutely well maintained. I believe that leads to the fact that many people enjoy Disney World, and they want the data to be accurate. Assuming the duplicate nodes can be pruned in such a way that there exists only one node for each location, would be ideal.

In [245]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'DataWranglingMongoDB-BiggerDataSet.ipynb'])

0